In [1]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

model.load_state_dict(torch.load('best_model.pth'))

device = torch.device('cuda')
model = model.to(device)

ModuleNotFoundError: No module named 'torch'

In [2]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

NameError: name 'torchvision' is not defined

In [3]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider]))

ModuleNotFoundError: No module named 'jetbot'

In [4]:

from jetbot import Robot

robot = Robot()

ModuleNotFoundError: No module named 'jetbot'

In [ ]:
import torch.nn.functional as F
import time

image_ct = 0
def update(change):
    global blocked_slider, robot, image_ct
    x = change['new'] 
    cv2.imwrite('./images_folder/' + str(image_ct) + '.png', x) 
    image_ct += 1
    x = preprocess(x)
    y = model(x)
    
    
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.5:
        robot.forward(0.4)
    else:
        robot.left(0.4)
    
    time.sleep(0.001)
        
update({'new': camera.value})  

In [5]:
camera.observe(update, names='value') 

NameError: name 'camera' is not defined

In [6]:
camera.observe(update, names='value') 
camera.unobserve(update, names='value')
robot.stop()
robot = Robot()
robot.stop()

NameError: name 'robot' is not defined